## Prep

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### dataset

In [2]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
# read and view
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print('length of datasets in characters: ', len(text))

length of datasets in characters:  1115394


In [5]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [6]:
# 统计出现的字符种类
chars = sorted(list(set(text)))     # 集合->列表->排序
vocab_size = len(chars)
print(''.join(chars))               # 列表元素拼接
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### tokenization (character-level)
> 对字符编码，用int表示每个字符（哈希）

In [7]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]           # 编码器：将字符串转换为编号的函数
decode = lambda l: ''.join([itos[i] for i in l])  # 解码器：将编号转换为字符串的函数

In [8]:
print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
data = torch.tensor(encode(text), dtype = torch.long)   # 编码整个数据集
print(data.shape, data.dtype)
print(data[:1000])  # 查看前1000个字符的编码结果

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# split into train and valid sets
n = int(0.9*len(data))   # train size(90%)
train_data = data[:n]
valid_data = data[n:]

In [11]:
block_size = 8      # 训练只会一块一块喂数据，而不是将整个数据集一次性全部放进模型
train_data[:block_size+1]   # +1用来保证transformer有8个target

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
# 解释为什么要+1: 每次输入对应一个输出token
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target} ')

when input is tensor([18]) the target: 47 
when input is tensor([18, 47]) the target: 56 
when input is tensor([18, 47, 56]) the target: 57 
when input is tensor([18, 47, 56, 57]) the target: 58 
when input is tensor([18, 47, 56, 57, 58]) the target: 1 
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15 
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47 
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58 


In [13]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8      # the maximum context length for predictions

def get_batch(split):   # 从所选数据集提取batch split='train' or 'valid'
    # generate a small batch of data of inputs x and target y
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))   # 从[0, len(data)-block_size]中随机挑选batch_size个整数，组成一个向量
                                                                # 作用：ix就是一个batch中各个向量的起始下标，len(data)-block_size是为了防越界
    x = torch.stack([data[i:i+block_size] for i in ix])         # 将batch_size个长为block_size的向量堆叠，形成(batch_size, block_size)的Tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])     # 堆叠形成(batch_size, block_size)的Tensor,但数据需要向后移动一格
    return x, y     # 返回 (value, target)

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print('yb.shape')   
print(yb)

print('------')

for b in range(batch_size):     # 行号
    for t in range(block_size): # 列号
        context = xb[b, :t+1]   
        target = yb[b, t]
        print(f'when input is {context.tolist()} the target: {target}')


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
yb.shape
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1,

## Bigram Language Model
> 二元模型，只根据当前一个词预测下一个词。不考虑上下文信息

### model def

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # 创建embedding table, 每个词(token)对应一个向量, table(字典)中的每个词都有对应的向量
        # 每个token_embedding向量存放当前token的各种信息，向量越长，存放的信息就越丰富
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # 查表, 将词转换为向量（传入idx下标）, 查表得到的向量就是根据当前词所作出的预测结果
        logits = self.token_embedding_table(idx)    # (batch, time, channel)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)    # 修改shape, 方便送入cross_entropy
            targets = targets.view(B*T)     
            loss = F.cross_entropy(logits, targets)

        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):    # max_new_tokens表示需要生成多少词
        # idx is (B, T) tensor, 存放当前上下文的标签
        for _ in range(max_new_tokens):
            # 预测结果
            logits, loss = self(idx)
            # 提取最后一个预测，即下一个词的预测结果
            logits = logits[:, -1, :]   # (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)   # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # 从预测的概率中随机选一个，不是直接选概率最高的那个词
            # append sampled idx to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx  
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)     # batch=1, time=1
# m.generate(idx, max_new_tokens=100).shape
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


### train

In [15]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)  # Adam is much better than SGD

In [16]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step() 

print(loss.item())

2.5727508068084717


In [17]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


## Self-Attention

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

### the mathematical trick in self-attention（逐步平均）
> 为了模型能够考虑上下文的内容，采用最简单的办法：直接将前文全部累加

方法一：朴素实现

In [18]:
# consider the followint toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2       # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# xbow[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))   # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]  # (t, C) 即前t个词
        xbow[b, t] = torch.mean(xprev, 0)   # 将前t个词做平均（在第一个维度上）, 存入xbow的第t位

In [20]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [21]:
xbow[0]     # 每行代表x的当前行+之前行的平均

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

方法2：通过加权矩阵乘法实现矩阵的逐步平均

In [22]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))    # 下对角矩阵
a = a / torch.sum(a, 1, keepdim=True)   # 将a的每行元素取平均
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b       # a要在前
print('a=', a, '\nb=', b, '\nc=', c)

a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 
c= tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [23]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x          # wei要在前 (T, T) @ (B, T, C) -> (B, T, T) @ (B, T, C) = (B, T, C)
print(xbow[0])
print(xbow2[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [24]:
torch.allclose(xbow, xbow2, atol=1e-6)

True

方法三：Softmax得到加权矩阵

In [25]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))     # e^-inf=0, e^0=1
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
wei = F.softmax(wei, dim=-1)    # 每行作softmax
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [27]:
xbows = wei @ x
torch.allclose(xbow, xbow2, atol=1e-6)

True

### Self-Attention (single head)

self-attention和逐步平均的区别：
+ self-attention不是对所有位置的平均，每个位置都有不同的权重

+ 而逐步平均每个位置的权重都相等

In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [28]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32          # batch, time, channels
x = torch.randn(B, T, C)

a single head

In [37]:
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)
wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) -> (B, T, T)
wei[0]

tensor([[-1.4305e+00,  6.4422e-01, -1.1744e+00, -5.5395e-01, -7.0512e-01,
         -1.0098e+00,  1.0479e+00,  3.3958e-01],
        [ 1.6443e-01, -4.6023e-01, -4.5886e-01, -7.0787e-02, -5.4128e-01,
         -9.3317e-01, -1.4435e+00, -5.9637e-01],
        [ 9.0444e-02,  8.2590e-02, -1.8939e-01, -2.6346e-01,  7.4627e-01,
         -8.2155e-01,  3.8205e-01,  3.5495e-01],
        [ 2.7647e+00,  9.7476e-01,  1.2441e+00,  2.0282e-01,  8.0770e-01,
          2.2209e+00, -8.6276e-02, -4.4204e-01],
        [-8.7457e-01,  8.7014e-01, -3.8303e-01,  6.9875e-01,  9.6535e-02,
         -9.5013e-01, -2.5728e-01,  4.6673e-01],
        [ 2.0034e+00,  7.0723e-01,  5.5891e-01, -2.7194e-03,  1.7009e-03,
          1.5529e+00,  2.0245e-01,  3.6714e-02],
        [ 1.7722e+00,  1.0007e+00,  1.8546e-01,  5.4104e-01,  2.0109e-01,
         -2.8099e-01, -3.8662e-01, -1.0212e+00],
        [-4.1920e-02, -9.2352e-01,  3.6489e-01,  1.0163e+00, -3.2969e-01,
         -1.0183e+00, -7.7183e-01, -6.7318e-01]], grad_fn=<Select

In [39]:
# masking, 屏蔽当前词之后的所有词
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))     # e^-inf=0, e^0=1
# softmax修改权重数据分布
wei = F.softmax(wei, dim=-1)
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6513, 0.3487, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3639, 0.3610, 0.2751, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6837, 0.1142, 0.1494, 0.0528, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0632, 0.3618, 0.1033, 0.3048, 0.1669, 0.0000, 0.0000, 0.0000],
        [0.4138, 0.1132, 0.0976, 0.0557, 0.0559, 0.2638, 0.0000, 0.0000],
        [0.4149, 0.1918, 0.0849, 0.1211, 0.0862, 0.0532, 0.0479, 0.0000],
        [0.1260, 0.0522, 0.1892, 0.3630, 0.0945, 0.0475, 0.0607, 0.0670]],
       grad_fn=<SelectBackward0>)

In [40]:
value = nn.Linear(C, head_size, bias=False)
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 32])

#### single-head attention layer

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [2]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        # QKV权重矩阵
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

### Multi-head Attention

+ 将整个embedding均匀分配到每个head上，而不仅仅用单独一个head来做attention，使得每个head关注输入的不同特征

+ 每个head大小head_size = n_embed // n_head

In [4]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])     # 多个head组成一个list
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)     # 把每个head的输出拼接起来
        out = self.dropout(self.proj(out))
        return out

### Transformer Block
+ decoder only

In [5]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x